# IESB - Graduacao - CIA028 - Costa Rica

In [ ]:
#Importando Bibliotecas
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.ensemble import RandomForestClassifier
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Carregando os dados
df = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')

df.shape, test.shape

In [ ]:
# Juntando os dataframes
df_all = df.append(test)

df_all.shape

In [ ]:
# Verificando tamanhos e tipos
df_all.info()

In [ ]:
# Quais colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

In [ ]:
# Olhando a coluna dependency
df_all['dependency'].value_counts()

In [ ]:
# Analisando os dados da coluna edjefa
df_all['edjefa'].value_counts()

In [ ]:
# Analisando os dados da coluna edjefe
df_all['edjefe'].value_counts()

In [ ]:
# Vamos transformar 'yes' em 1 e 'no' em 0
# nas colunas edjefa e edjefe
mapeamento = {'yes': 1, 'no': 0}

df_all['edjefa'] = df_all['edjefa'].replace(mapeamento).astype(int)
df_all['edjefe'] = df_all['edjefe'].replace(mapeamento).astype(int)

In [ ]:
# Quais colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

In [ ]:
# Olhando a coluna dependency
df_all['dependency'].value_counts()

In [ ]:
# Vamos transformar 'yes' em 1 e 'no' em 0
# na coluna dependency
df_all['dependency'] = df_all['dependency'].replace(mapeamento).astype(float)

In [ ]:
# Quais colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

In [ ]:
# Visualizando do comando info
df_all.info()

In [ ]:
# Verificando os valores nulos
df_all.isnull().sum()

In [ ]:
 # Verificando os valores de aluguel (v2a1) para os chefes/as de familia (parentesco1 = 1)
df_all[df_all['parentesco1'] == 1]['v2a1'].isnull().sum()

In [ ]:
# Qual a cara dos dados de v18q
df_all['v18q'].value_counts()

In [ ]:
# Prenchendo com -1 os valores nulos de v2a1
df_all['v2a1'].fillna(-1, inplace=True)

In [ ]:
# Prenchendo com 0 os valores nulos de v18q1
df_all['v18q1'].fillna(0, inplace=True)

In [ ]:
# Verificando os valores nulos
df_all.isnull().sum().sort_values()

In [ ]:
# Prenchendo com -1 os valores nulos de SQBmeaned, meaneduc e rez_esc
df_all['SQBmeaned'].fillna(-1, inplace=True)
df_all['meaneduc'].fillna(-1, inplace=True)
df_all['rez_esc'].fillna(-1, inplace=True)

In [ ]:
# Separando as colunas para treinamento
feats = [c for c in df_all.columns if c not in ['Id', 'idhogar', 'Target']]

In [ ]:
# Separar os dataframes
train, test = df_all[~df_all['Target'].isnull()], df_all[df_all['Target'].isnull()]

train.shape, test.shape

# Copiando do campeão

# Pontuação - 0.43687


rf = RandomForestClassifier(max_depth=None, random_state=42, n_jobs=4, n_estimators=700,
                            min_impurity_decrease=1e-3, min_samples_leaf=2,
                            verbose=0, class_weight='balanced')

rf.fit(train[feats], train['Target'])

test['Target'] = rf.predict(test[feats]).astype(int)
test[['Id', 'Target']].to_csv('submission.csv', index=False)

In [ ]:
# Modificação 01

#Realizei um ajuste mais fino no número de árvores e consegui subir um pouco a pontuação.

#Pontuação - 43746  



rf1 = RandomForestClassifier(max_depth=None, random_state=42, n_jobs=4, n_estimators=701,
                            min_impurity_decrease=1e-3, min_samples_leaf=2,
                            verbose=0, class_weight='balanced')

rf1.fit(train[feats], train['Target'])

test['Target'] = rf1.predict(test[feats]).astype(int)
test[['Id', 'Target']].to_csv('submission.csv', index=False)

# Modificação 02

#Adcionei o parâmetro min_samples_split, que seta um número de amostras mínimas para divisão de um nó, e consegui subir a pontuação.

#Pontuação - 0.43821  



rf2 = RandomForestClassifier(max_depth=None, random_state=42, n_jobs=4, n_estimators=701,
                            min_impurity_decrease=1e-3, min_samples_leaf=2, min_samples_split=5,
                            verbose=0, class_weight='balanced')

rf2.fit(train[feats], train['Target'])

test['Target'] = rf2.predict(test[feats]).astype(int)
test[['Id', 'Target']].to_csv('submission.csv', index=False)

# Modificação 03

#Ajustei o valor do parâmetro min_impurity_decrease, que divide um nó, caso essa divisão reduza a impureza, baseado no valor setado. Com isso consegui subir a pontuação

#Pontuação - 0.44049



rf3 = RandomForestClassifier(max_depth=None, random_state=42, n_jobs=4, n_estimators=701,
                            min_impurity_decrease=0.0007, min_samples_leaf=2, min_samples_split=5,
                            verbose=0, class_weight='balanced')

rf3.fit(train[feats], train['Target'])

test['Target'] = rf3.predict(test[feats]).astype(int)
test[['Id', 'Target']].to_csv('submission.csv', index=False)